In [1]:
%pip install psycopg2

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 74.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import psycopg2

password = getpass.getpass("Enter the DB Password")

connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password=password,
    database="manager"
)

cursor = connection.cursor()

create_investment_table = """
create table investment(
    id serial primary key,
    coin varchar(32),
    currency varchar(3),
    amount real
)
"""
# "serial" data type is unique postgres data type that gives a column a unique integer numbers i.e. IDs that auto increment for each row. so id column is a "serial" type
# "real" is python equivalent of floay data type.

cursor.execute(create_investment_table)

connection.commit()

cursor.close()
connection.close()

In [3]:
#Inserting data to the Database
import getpass
import psycopg2

password = getpass.getpass("Enter the DB Password")

connection = psycopg2.connect(
    host="localhost",
    user="postgres",
    password=password,
    database="manager"
)

cursor = connection.cursor()

create_investment_table = """
create table if not exists investment(
    id serial primary key,
    coin varchar(32),
    currency varchar(3),
    amount real
)
"""
# "serial" data type is unique postgres data type that gives a column a unique integer numbers i.e. IDs that auto increment for each row. so id column is a "serial" type
# "real" is python equivalent of floay data type.

cursor.execute(create_investment_table)

# add_bitcoin_investment="""
# insert into investment (
#     coin, currency, amount
# ) values ('bitcoin', 'usd', 1.0); These are hard coded values, see below to dynamically assign values
# """
# cursor.execute(add_bitcoin_investment)

add_bitcoin_investment="""
insert into investment (
    coin, currency, amount
) values %s;
"""# %s is how you parameterize a sql query for psycopg2 based queries. Query with %s or any other placeholder is also called SQL template.

data = [
    ("etherium", "GBP", 10.0),
    ("bitcoin", "USD", 1.0),
    ("dogecoin", "INR", 20.0)
    # Add more Row data as tuples    
]
import psycopg2.extras #extras module has execute with values function that usues SQL template with values.

psycopg2.extras.execute_values(cursor,add_bitcoin_investment,data)


connection.commit()

# cursor.close()
# connection.close()

In [9]:
#retrieve data
select_bitcoin_investments = "select * from investment where coin='bitcoin';"

cursor.execute(select_bitcoin_investments)

cursor.fetchall()


[RealDictRow([('id', 1),
              ('coin', 'bitcoin'),
              ('currency', 'usd'),
              ('amount', 1.0)]),
 RealDictRow([('id', 3),
              ('coin', 'bitcoin'),
              ('currency', 'USD'),
              ('amount', 1.0)]),
 RealDictRow([('id', 6),
              ('coin', 'bitcoin'),
              ('currency', 'USD'),
              ('amount', 1.0)]),
 RealDictRow([('id', 9),
              ('coin', 'bitcoin'),
              ('currency', 'USD'),
              ('amount', 1.0)])]

In [ ]:
#So just like row_factories in sqlite, we have cursor factories in the psycopg2 postgres library. It will enable cursor to use a function to return a python object when querying the data.

#Let's see a inbuilt cursor factory wihtin the extras module

cursor = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)

#retrieve data
select_bitcoin_investments = "select * from investment where coin='bitcoin';"

cursor.execute(select_bitcoin_investments)

cursor.fetchall() #Not exactly a dictionary but a list of tuples.

[RealDictRow([('id', 1),
              ('coin', 'bitcoin'),
              ('currency', 'usd'),
              ('amount', 1.0)]),
 RealDictRow([('id', 3),
              ('coin', 'bitcoin'),
              ('currency', 'USD'),
              ('amount', 1.0)]),
 RealDictRow([('id', 6),
              ('coin', 'bitcoin'),
              ('currency', 'USD'),
              ('amount', 1.0)]),
 RealDictRow([('id', 9),
              ('coin', 'bitcoin'),
              ('currency', 'USD'),
              ('amount', 1.0)])]

In [ ]:
#Trying to get an actual list of dictionary. With column names as keys.
cursor.execute(select_bitcoin_investments)

data = [dict(row) for row in cursor.fetchall()] #List comprehension that casts RealDictRow types into dict types. Each tuple becomes a key value pair.

print(data)

[{'id': 1, 'coin': 'bitcoin', 'currency': 'usd', 'amount': 1.0}, {'id': 3, 'coin': 'bitcoin', 'currency': 'USD', 'amount': 1.0}, {'id': 6, 'coin': 'bitcoin', 'currency': 'USD', 'amount': 1.0}, {'id': 9, 'coin': 'bitcoin', 'currency': 'USD', 'amount': 1.0}]


In [26]:
#Let's use dataclass to parse the query output into a python object just like we did in sqlite

from dataclasses import dataclass

@dataclass
class Investment:
    id: int
    coin: str
    currency: str
    amount: float
    
cursor.execute(select_bitcoin_investments)

data = [Investment(**dict(row)) for row in cursor.fetchall()] #passing the type casted dictionary as multiple keyword arguments can instantiate the investment class and give investment objects

#Now data is a list of Investment objects

for investment in data:
    print(f"ID: {investment.id} ===== {investment.coin} Coin has been found. Amount is {investment.amount} and currency that it was traded in was {investment.currency}")

ID: 1 ===== bitcoin Coin has been found. Amount is 1.0 and currency that it was traded in was usd
ID: 3 ===== bitcoin Coin has been found. Amount is 1.0 and currency that it was traded in was USD
ID: 6 ===== bitcoin Coin has been found. Amount is 1.0 and currency that it was traded in was USD
ID: 9 ===== bitcoin Coin has been found. Amount is 1.0 and currency that it was traded in was USD


In [ ]:
#Now let's make the Cryptocurrency manager tool use postgres

#Implementation with Cursor factory
import psycopg2.extras
import requests
import click
import psycopg2
import csv
from dataclasses import dataclass

api_key = "CG-YgaSDYyvsiDQsZvTB8KNjJcR"

headers = {
    "x-cg-demo-api-key": api_key
}

#Data class to define a data structure for the Investments table row.
@dataclass
class Investment:
    id: int
    coin: str
    currency: str
    amount: float
    

def get_connection():
    connection = psycopg2.connect(
        host="localhost",
        user="postgres",
        password="Test@1234",
        database="manager"
    )
    
    return connection

#Create a command group, a group can have multiple commands assocated with it. The group name will the function name
@click.group()
def cli():
    pass

@cli.command()
@click.option("--coin") #Adding an option
@click.option("--currency")
@click.option("--amount")
def new_investment(coin, currency, amount):
    sql = f"""
    insert into investment (
        coin, currency, amount
    ) values (
        '{coin.lower()}', '{currency.lower()}', {amount}
    );
    """
    connection = get_connection()
    cursor = connection.cursor()
    
    cursor.execute(sql)
    
    connection.commit()
    
    cursor.close()
    connection.close()
    
@cli.command()
@click.option("--filename")
def import_investments(filename):
    with open(filename, "r") as f:
        file_reader = csv.reader(f)
        rows = [[x.lower() for x in row[1:]] for row in file_reader] #we use row[1:], this means from each row just extract data from the second column. The first column of the CSV will be id.
        
    print(f"Addded {len(rows)} of investments into the table")
    
    sql = "insert into investment (coin, currency, amount) values %s"
    
    connection = get_connection()
    cursor = connection.cursor()
    
    psycopg2.extras.execute_values(cursor, sql, rows)
    
    connection.commit()
    
    cursor.close()
    connection.close()


def get_coin_price(coin_id, currency):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={coin_id}&vs_currencies={currency}"
    data = requests.get(url, headers=headers).json()
    return data[coin_id][currency]

@cli.command()
@click.option("--currency")
def view_investments(currency):
    connection = get_connection()
    cursor = connection.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    if currency:
        sql = f"select * from investment where currency={currency.lower()}"
    else:
        sql = "select * from investment"
    
    cursor.execute(sql)
    
    data = [Investment(**dict(row)) for row in cursor.fetchall()]
    
    cursor.close()
    connection.close()   
    
    for investment in data:
        print(f"{investment.amount} {investment.coin} in {investment.currency.lower()} is today worth {investment.amount * get_coin_price(investment.coin, investment.currency.lower())}")
    

# @cli.command() #Command decorator that aslo assigns this command to the group.
# @click.option("--coin_id", default="bitcoin") #Adding an option
# @click.option("--currency", default="usd")
# def get_investment_value(coin_id, currency): #the option name gets passed onto the function parameter name. Then the function uses iut wthin the code.
#     coin_price = get_coin_price(coin_id, currency) #Now we can select all the columns and not amounts as we have a data factory
#     sql = """SELECT *
#     FROM investments
#     WHERE coin_id=?
#     AND currency=?
#     AND sell=?;"""
#     buy_result = cursor.execute(sql, (coin_id, currency, False)).fetchall()
#     sell_result = cursor.execute(sql, (coin_id, currency, True)).fetchall()
    
#     buy_amount = sum(row.amount for row in buy_result) #With data factory implemented we now get row Investmeent objects that has 'amount' attribute. So now we can use dot notation.
#     sell_amount = sum(row.amount for row in sell_result)
    
#     total = buy_amount - sell_amount
    
#     print(f"You own a total of {total} {coin_id} worth {total * coin_price} {currency.upper()}")

# @cli.command()
# @click.option("--coin_id", default="bitcoin")
# @click.option("--currency", default="usd")
# def show_coin_price(coin_id, currency):
#     coin_price = get_coin_price(coin_id, currency)
#     print(f"The price of {coin_id} is {coin_price:.2f} {currency.upper()}")
    
# @cli.command()
# @click.option("--coin_id")
# @click.option("--currency")
# @click.option("--amount", type=float)
# @click.option("--sell", is_flag=True)
# def add_investments(coin_id, currency, amount, sell):
#     try:
#         sql = "INSERT INTO investments (coin_id, currency, amount, sell, date) VALUES (?, ?, ?, ?, ?);"
#         values = (coin_id, currency, amount, int(sell), datetime.datetime.now())
#         cursor.execute(sql, values)
#         database.commit()
#         print(f"Added {'sell' if sell else 'buy'} of {amount} {coin_id}")
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         database.rollback()

# Function to run commands in Jupyter
def run_command(command, *args):
    try:
        cli.main(args=[command] + list(args), standalone_mode=False)
    except SystemExit:
        pass

# Example usage in Jupyter
# run_command('import-investments', '--filename', 'import.csv')
# run_command('new-investment', '--coin', 'ripple', '--currency', 'inr', '--amount', '1000.0')
run_command('view-investments')

1.0 bitcoin in usd is today worth 105440.0


KeyError: 'USD'